In [1]:
import pandas as pd
import numpy as np

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

from tqdm import tnrange, tqdm_notebook as tqdm
import time
import random
from copy import deepcopy

import pickle

In [156]:
df = pd.read_pickle('bgg_full_df_1200.pkl')

In [157]:
df.head()

,name,overall_rank,category_rank,no_of_players,playing_time,complexity,prices,categories,mechanism
0,7.1\nThurn and Taxis (2006)\n17K Ratings & 3.9...,OVERALL 402,STRATEGY 281,2–4 Players\nCommunity: 2–4 — Best: 3–4,60 Min\nPlaying Time,Weight: 2.26 / 5\nComplexity Rating\n'Complexi...,STORES\n(unavailable)\nAmazon.com\nEbay listings,Categories\nPost-Napoleonic\nTransportation,Mechanisms\nCard Drafting\nHand Management\nNe...
1,7.5\nCosmic Encounter (2008)\n26K Ratings & 5....,OVERALL 130,THEMATIC 50,3–5 Players\nCommunity: 4–5+ — Best: 5,60–120 Min\nPlaying Time,Weight: 2.57 / 5\nComplexity Rating\n'Complexi...,STORES\n$59.95\nSuggested retail\n$53.95\nAmaz...,Categories\nBluffing\nNegotiation\nScience Fic...,Mechanisms\nAlliances\nAuction/Bidding\nHand M...
2,7.1\nEminent Domain (2011)\n9K Ratings & 2.3K ...,OVERALL 515,STRATEGY 334,2–4 Players\nCommunity: 2–4 — Best: 3,45 Min\nPlaying Time,Weight: 2.50 / 5\nComplexity Rating\n'Complexi...,STORES\n$39.99\nSuggested retail\n$29.82\nAmaz...,Categories\nCard Game\nCivilization\nScience F...,"Mechanisms\nCard Drafting\nDeck, Bag, and Pool..."
3,7.9\nYokohama (2016)\n7.4K Ratings & 1.4K Comm...,OVERALL 93,STRATEGY 65,2–4 Players\nCommunity: 2–4 — Best: 3,90 Min\nPlaying Time,Weight: 3.31 / 5\nComplexity Rating\n'Complexi...,STORES\n$59.95\nSuggested retail\n$51.61\nAmaz...,Category\nEconomic,Mechanisms\nContracts\nEnd Game Bonuses\nGrid ...
4,8.3\nGreat Western Trail (2016)\n24K Ratings &...,OVERALL 10,STRATEGY 10,2–4 Players\nCommunity: 2–4 — Best: 3,75–150 Min\nPlaying Time,Weight: 3.70 / 5\nComplexity Rating\n'Complexi...,STORES\n(unavailable)\nAmazon.com\nEbay listings,Categories\nAmerican West\nAnimals,"Mechanisms\nDeck, Bag, and Pool Building\nHand..."


In [158]:
names = df['name'].str.split('\n', expand =True)

In [159]:
df = names.merge(df, how='inner', left_index=True, right_index=True)

In [160]:
df['overall_rank'] = df['overall_rank'].str.split(' ', expand =True)[1]

In [161]:
df['category_rank_cat'] = df['category_rank'].str.split(' ', expand= True)[0]

In [162]:
df['category_rank'] = df['category_rank'].str.split(' ', expand= True)[1]

In [163]:
df['no_of_players_suggested'] = df['no_of_players'].str.split('\n', expand =True)[0]

In [164]:
df['no_of_players_best_rec']= df['no_of_players'].str.split('\n', expand =True)[1].str.split('Best: ', expand =True)[1]

In [165]:
df['playing_time'] = df['playing_time'].str.split('\n', expand = True)[0]

In [166]:
df['complexity'] = df['complexity'].str.split('\n', expand =True)[0]

In [167]:
df['categories']= df['categories'].str.split('\n')

In [168]:
df['categories']

0           [Categories, Post-Napoleonic, Transportation]
1       [Categories, Bluffing, Negotiation, Science Fi...
2       [Categories, Card Game, Civilization, Science ...
3                                    [Category, Economic]
4                    [Categories, American West, Animals]
                              ...                        
1194                        [Categories, Pirates, Racing]
1195       [Categories, Abstract Strategy, City Building]
1196    [Categories, American Indian Wars, Economic, F...
1197                [Categories, City Building, Medieval]
1198    [Categories, Adventure, Exploration, Fantasy, ...
Name: categories, Length: 1199, dtype: object

In [172]:
df['categories'] = df['categories'].apply(lambda x: x.remove(('Category')) if ('Category') in x else x)

In [177]:
df['categories'].isna().sum()

0

In [176]:
df['categories'] = df['categories'].apply(lambda x: x.remove(('Categories')) if ('Categories') in x else x)

In [197]:
df['mechanism'] = df['mechanism'].str.split('\n')

In [178]:
df['overall_rank'] = df['overall_rank'].str.replace(',', '')

In [179]:
df['category_rank'] = df['category_rank'].str.replace(',', '')

In [180]:
df['overall_rank'] = pd.to_numeric(df['overall_rank'])

In [181]:
df['category_rank'] = pd.to_numeric(df['category_rank'])

In [182]:
df = df.sort_values(by='overall_rank')

In [198]:
df['mechanism']

577     [Mechanisms, Action Retrieval, Campaign / Batt...
373     [Mechanisms, Action Points, Cooperative Game, ...
356     [Mechanisms, Card Drafting, End Game Bonuses, ...
27      [Mechanisms, Connections, Hand Management, Inc...
459     [Mechanisms, Action Points, Auction/Bidding, A...
                              ...                        
847     [Mechanisms, Hidden Movement, Point to Point M...
983                         [Mechanism, Cooperative Game]
754     [Mechanisms, Dice Rolling, Hexagon Grid, Varia...
1164    [Mechanisms, Dice Rolling, Hand Management, Pi...
896     [Mechanisms, Hand Management, Hidden Roles, Ki...
Name: mechanism, Length: 1199, dtype: object

In [201]:
df['mechanism'] = df['mechanism'].apply(lambda x: x.remove(('Mechanisms')) if 'Mechanisms' in x else x)

In [204]:
df['mechanism'] = df['mechanism'].apply(lambda x: x.remove(('Mechanism')) if 'Mechanism' in x else x)

In [222]:
df['complexity'] = df['complexity'].apply(lambda x: x[8:12])

In [226]:
df['complexity'] = pd.to_numeric(df['complexity'])

In [234]:
df.head()

,0,1,2,name,overall_rank,category_rank,no_of_players,playing_time,complexity,prices,categories,mechanism,category_rank_cat,no_of_players_suggested,no_of_players_best_rec
577,8.8,Gloomhaven (2017),33K Ratings & 6.3K Comments · GeekBuddy Analysis,8.8\nGloomhaven (2017)\n33K Ratings & 6.3K Com...,1,1.0,1–4 Players\nCommunity: 1–4 — Best: 3,60–120 Min,3.82,STORES\n$140.00\nSuggested retail\n$101.83\nAm...,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Retrieval, Campaign / Battle Card Driv...",THEMATIC,1–4 Players,3
373,8.6,Pandemic Legacy: Season 1 (2015),36K Ratings & 5.6K Comments · GeekBuddy Analysis,8.6\nPandemic Legacy: Season 1 (2015)\n36K Rat...,2,2.0,2–4 Players\nCommunity: 2–4 — Best: 4,60 Min,2.83,STORES\n$69.99\nSuggested retail\n$56.50\nAmaz...,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...",THEMATIC,2–4 Players,4
356,8.4,Terraforming Mars (2016),52K Ratings & 7.7K Comments · GeekBuddy Analysis,8.4\nTerraforming Mars (2016)\n52K Ratings & 7...,3,5.0,1–5 Players\nCommunity: 1–4 — Best: 3,120 Min,3.23,STORES\n$69.95\nSuggested retail\n$41.99\nAmaz...,"[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem...",STRATEGY,1–5 Players,3
27,8.6,Brass: Birmingham (2018),12K Ratings & 2K Comments · GeekBuddy Analysis,8.6\nBrass: Birmingham (2018)\n12K Ratings & 2...,4,3.0,2–4 Players\nCommunity: 2–4 — Best: 3–4,60–120 Min,3.92,STORES\n$76.00\nAmazon.com\nFrom £64.67\nEbay ...,"[Economic, Industry / Manufacturing, Transport...","[Connections, Hand Management, Income, Loans, ...",STRATEGY,2–4 Players,3–4
459,8.5,Through the Ages: A New Story of Civilization ...,19K Ratings & 2.8K Comments · GeekBuddy Analysis,8.5\nThrough the Ages: A New Story of Civiliza...,5,4.0,2–4 Players\nCommunity: 2–4 — Best: 3,120 Min,4.39,STORES\n$69.95\nSuggested retail\n$52.81\nAmaz...,"[Card Game, Civilization, Economic]","[Action Points, Auction/Bidding, Auction: Dutc...",STRATEGY,2–4 Players,3


In [236]:
df.drop(columns = ['name', 'no_of_players'], inplace=True)

In [245]:
df['year'] = df[1].apply(lambda x: x[-7:])

In [247]:
df['name'] = df[1].apply(lambda x: x[:-7])

In [251]:
df['name']= df['name'].str.strip()

In [253]:
df.columns

Index([                        0,                         1,
                               2,            'overall_rank',
                 'category_rank',            'playing_time',
                    'complexity',                  'prices',
                    'categories',               'mechanism',
             'category_rank_cat', 'no_of_players_suggested',
        'no_of_players_best_rec',                    'year',
                          'name'],
      dtype='object')

In [255]:
df.head()

,0,1,2,overall_rank,category_rank,playing_time,complexity,prices,categories,mechanism,category_rank_cat,no_of_players_suggested,no_of_players_best_rec,year,name
577,8.8,Gloomhaven (2017),33K Ratings & 6.3K Comments · GeekBuddy Analysis,1,1.0,60–120 Min,3.82,STORES\n$140.00\nSuggested retail\n$101.83\nAm...,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Retrieval, Campaign / Battle Card Driv...",THEMATIC,1–4 Players,3,(2017),Gloomhaven
373,8.6,Pandemic Legacy: Season 1 (2015),36K Ratings & 5.6K Comments · GeekBuddy Analysis,2,2.0,60 Min,2.83,STORES\n$69.99\nSuggested retail\n$56.50\nAmaz...,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...",THEMATIC,2–4 Players,4,(2015),Pandemic Legacy: Season 1
356,8.4,Terraforming Mars (2016),52K Ratings & 7.7K Comments · GeekBuddy Analysis,3,5.0,120 Min,3.23,STORES\n$69.95\nSuggested retail\n$41.99\nAmaz...,"[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem...",STRATEGY,1–5 Players,3,(2016),Terraforming Mars
27,8.6,Brass: Birmingham (2018),12K Ratings & 2K Comments · GeekBuddy Analysis,4,3.0,60–120 Min,3.92,STORES\n$76.00\nAmazon.com\nFrom £64.67\nEbay ...,"[Economic, Industry / Manufacturing, Transport...","[Connections, Hand Management, Income, Loans, ...",STRATEGY,2–4 Players,3–4,(2018),Brass: Birmingham
459,8.5,Through the Ages: A New Story of Civilization ...,19K Ratings & 2.8K Comments · GeekBuddy Analysis,5,4.0,120 Min,4.39,STORES\n$69.95\nSuggested retail\n$52.81\nAmaz...,"[Card Game, Civilization, Economic]","[Action Points, Auction/Bidding, Auction: Dutc...",STRATEGY,2–4 Players,3,(2015),Through the Ages: A New Story of Civilization


In [254]:
df[['name', 'year','overall_rank', 'category_rank_cat','category_rank', 'complexity','playing_time','categories','mechanism', 'no_of_players_suggested', 'no_of_players_best_rec','prices']]

,name,year,overall_rank,category_rank_cat,category_rank,complexity,playing_time,categories,mechanism,no_of_players_suggested,no_of_players_best_rec,prices
577,Gloomhaven,(2017),1,THEMATIC,1.0,3.82,60–120 Min,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Retrieval, Campaign / Battle Card Driv...",1–4 Players,3,STORES\n$140.00\nSuggested retail\n$101.83\nAm...
373,Pandemic Legacy: Season 1,(2015),2,THEMATIC,2.0,2.83,60 Min,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...",2–4 Players,4,STORES\n$69.99\nSuggested retail\n$56.50\nAmaz...
356,Terraforming Mars,(2016),3,STRATEGY,5.0,3.23,120 Min,"[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem...",1–5 Players,3,STORES\n$69.95\nSuggested retail\n$41.99\nAmaz...
27,Brass: Birmingham,(2018),4,STRATEGY,3.0,3.92,60–120 Min,"[Economic, Industry / Manufacturing, Transport...","[Connections, Hand Management, Income, Loans, ...",2–4 Players,3–4,STORES\n$76.00\nAmazon.com\nFrom £64.67\nEbay ...
459,Through the Ages: A New Story of Civilization,(2015),5,STRATEGY,4.0,4.39,120 Min,"[Card Game, Civilization, Economic]","[Action Points, Auction/Bidding, Auction: Dutc...",2–4 Players,3,STORES\n$69.95\nSuggested retail\n$52.81\nAmaz...
...,...,...,...,...,...,...,...,...,...,...,...,...
847,Scotland Yard,(1983),1196,FAMILY,386.0,1.96,45 Min,"[Deduction, Travel]","[Hidden Movement, Point to Point Movement, Sec...",3–6 Players,none,STORES\n$79.99\nAmazon.com\nFrom $12.50\nEbay ...
983,Exit: The Game – The Sunken Treasure,(2017),1197,None,NaN,1.72,45–90 Min,[Deduction],[Cooperative Game],1–4 Players,2,STORES\n$17.89\nWalmart.com\nEbay listings
754,Ogre,(1977),1198,WAR,251.0,2.35,45 Min,"[Fighting, Science Fiction, Wargame]","[Dice Rolling, Hexagon Grid, Variable Player P...",1–3 Players,2,STORES\nEbay listings
1164,Pirate's Cove,(2002),1199,FAMILY,376.0,2.01,60–90 Min,"[Adventure, Fighting, Nautical, Pirates]","[Dice Rolling, Hand Management, Pick-up and De...",3–5 Players,5,STORES\n$174.99\nAmazon.com\nEbay listings


In [228]:
cooperative = df[df['mechanism'].apply(lambda x: 'Cooperative Game' in x)]

In [233]:
cooperative[cooperative['complexity'].apply(lambda x: x>3)]

,0,1,2,name,overall_rank,category_rank,no_of_players,playing_time,complexity,prices,categories,mechanism,category_rank_cat,no_of_players_suggested,no_of_players_best_rec
577,8.8,Gloomhaven (2017),33K Ratings & 6.3K Comments · GeekBuddy Analysis,8.8\nGloomhaven (2017)\n33K Ratings & 6.3K Com...,1,1.0,1–4 Players\nCommunity: 1–4 — Best: 3,60–120 Min,3.82,STORES\n$140.00\nSuggested retail\n$101.83\nAm...,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Retrieval, Campaign / Battle Card Driv...",THEMATIC,1–4 Players,3
317,8.3,Spirit Island (2017),18K Ratings & 3.1K Comments · GeekBuddy Analysis,8.3\nSpirit Island (2017)\n18K Ratings & 3.1K ...,13,12.0,1–4 Players\nCommunity: 1–4 — Best: 2,90–120 Min,3.96,STORES\n$79.95\nSuggested retail\n$52.84\nAmaz...,"[Age of Reason, Environmental, Fantasy, Fighti...","[Action Retrieval, Area Majority / Influence, ...",STRATEGY,1–4 Players,2
147,8.2,Arkham Horror: The Card Game (2016),22K Ratings & 3.6K Comments · GeekBuddy Analysis,8.2\nArkham Horror: The Card Game (2016)\n22K ...,20,1.0,1–2 Players\nCommunity: 1–2+ — Best: 2,60–120 Min,3.37,STORES\n$39.95\nSuggested retail\n$35.93\nAmaz...,"[Adventure, Card Game, Collectible Components,...","[Action Points, Cooperative Game, Deck, Bag, a...",CUSTOMIZABLE,1–2 Players,2
148,8.1,Mage Knight Board Game (2011),26K Ratings & 5.3K Comments · GeekBuddy Analysis,8.1\nMage Knight Board Game (2011)\n26K Rating...,25,11.0,1–4 Players\nCommunity: 1–3 — Best: 1–2,60–240 Min,4.29,STORES\n$124.99\nSuggested retail\n$101.95\nAm...,"[Adventure, Exploration, Fantasy, Fighting]","[Card Drafting, Cooperative Game, Deck, Bag, a...",THEMATIC,1–4 Players,1–2
200,8.7,Kingdom Death: Monster (2015),6.8K Ratings & 1.6K Comments · GeekBuddy Analysis,8.7\nKingdom Death: Monster (2015)\n6.8K Ratin...,32,6.0,1–4 Players\nCommunity: 1–4 — Best: 4,60–180 Min,4.23,STORES\nEbay listings,"[Adventure, Fantasy, Horror, Mature / Adult, M...","[Cooperative Game, Dice Rolling, Grid Movement...",THEMATIC,1–4 Players,4
61,8.2,Pandemic Legacy: Season 2 (2017),9.5K Ratings & 1.6K Comments · GeekBuddy Analysis,8.2\nPandemic Legacy: Season 2 (2017)\n9.5K Ra...,33,8.0,2–4 Players\nCommunity: 2–4 — Best: 4,60 Min,3.24,STORES\n$79.99\nSuggested retail\n$63.98\nAmaz...,"[Environmental, Exploration, Medical]","[Cooperative Game, Hand Management, Legacy Gam...",THEMATIC,2–4 Players,4
256,8.5,Nemesis (2018),7.4K Ratings & 1.6K Comments · GeekBuddy Analysis,8.5\nNemesis (2018)\n7.4K Ratings & 1.6K Comme...,36,9.0,1–5 Players\nCommunity: 1–5 — Best: 4,90–180 Min,3.30,STORES\n$299.98\nAmazon.com\nFrom $699.95\nEba...,"[Adventure, Horror, Miniatures, Science Fiction]","[Campaign / Battle Card Driven, Cooperative Ga...",THEMATIC,1–5 Players,4
481,7.9,Robinson Crusoe: Adventures on the Cursed Isla...,30K Ratings & 5.2K Comments · GeekBuddy Analysis,7.9\nRobinson Crusoe: Adventures on the Cursed...,46,15.0,1–4 Players\nCommunity: 1–4 — Best: none,60–120 Min,3.76,STORES\n$65.00\nSuggested retail\n$45.92\nAmaz...,"[Adventure, Exploration, Fighting, Novel-based]","[Action Queue, Cooperative Game, Dice Rolling,...",THEMATIC,1–4 Players,none
316,7.8,Eldritch Horror (2013),26K Ratings & 4.2K Comments · GeekBuddy Analysis,7.8\nEldritch Horror (2013)\n26K Ratings & 4.2...,63,18.0,1–8 Players\nCommunity: 1–6 — Best: 4,120–240 Min,3.31,STORES\n$59.99\nSuggested retail\n$44.99\nAmaz...,"[Adventure, Fantasy, Fighting, Horror, Novel-b...","[Action Points, Cooperative Game, Dice Rolling...",THEMATIC,1–8 Players,4
234,8.4,Too Many Bones (2017),4.2K Ratings & 956 Comments · GeekBuddy Analysis,8.4\nToo Many Bones (2017)\n4.2K Ratings & 956...,71,12.0,1–4 Players\nCommunity: 1–4 — Best: 3,60–120 Min,3.82,STORES\nEbay listings,"[Adventure, Dice, Fantasy]","[Cooperative Game, Deck, Bag, and Pool Buildin...",THEMATIC,1–4 Players,3


In [132]:
df.shape

(1199, 15)

In [191]:
df.tail()

,0,1,2,name,overall_rank,category_rank,no_of_players,playing_time,complexity,prices,categories,mechanism,category_rank_cat,no_of_players_suggested,no_of_players_best_rec
847,6.5,Scotland Yard (1983),13K Ratings & 3.2K Comments · GeekBuddy Analysis,6.5\nScotland Yard (1983)\n13K Ratings & 3.2K ...,1196,386.0,3–6 Players\nCommunity: 2–6 — Best: none,45 Min,Weight: 1.96 / 5,STORES\n$79.99\nAmazon.com\nFrom $12.50\nEbay ...,"[Deduction, Travel]",Mechanisms\nHidden Movement\nPoint to Point Mo...,FAMILY,3–6 Players,none
983,7.2,Exit: The Game – The Sunken Treasure (2017),1.5K Ratings & 284 Comments · GeekBuddy Analysis,7.2\nExit: The Game – The Sunken Treasure (201...,1197,NaN,1–4 Players\nCommunity: 1–4 — Best: 2,45–90 Min,Weight: 1.72 / 5,STORES\n$17.89\nWalmart.com\nEbay listings,[Deduction],Mechanism\nCooperative Game,None,1–4 Players,2
754,6.9,Ogre (1977),2.6K Ratings & 1.4K Comments · GeekBuddy Analysis,6.9\nOgre (1977)\n2.6K Ratings & 1.4K Comments...,1198,251.0,1–3 Players\nCommunity: 1–3 — Best: 2,45 Min,Weight: 2.35 / 5,STORES\nEbay listings,"[Fighting, Science Fiction, Wargame]",Mechanisms\nDice Rolling\nHexagon Grid\nVariab...,WAR,1–3 Players,2
1164,6.6,Pirate's Cove (2002),7.8K Ratings & 2.5K Comments · GeekBuddy Analysis,6.6\nPirate's Cove (2002)\n7.8K Ratings & 2.5K...,1199,376.0,3–5 Players\nCommunity: 3–5 — Best: 5,60–90 Min,Weight: 2.01 / 5,STORES\n$174.99\nAmazon.com\nEbay listings,"[Adventure, Fighting, Nautical, Pirates]",Mechanisms\nDice Rolling\nHand Management\nPic...,FAMILY,3–5 Players,5
896,6.5,BANG! (2002),25K Ratings & 5.7K Comments · GeekBuddy Analysis,6.5\nBANG! (2002)\n25K Ratings & 5.7K Comments...,1200,156.0,4–7 Players\nCommunity: 5–7 — Best: 7,20–40 Min,Weight: 1.64 / 5,STORES\n$19.99\nSuggested retail\n$17.04\nAmaz...,"[American West, Bluffing, Card Game, Deduction...",Mechanisms\nHand Management\nHidden Roles\nKil...,PARTY,4–7 Players,7


In [190]:
df['no_of_players_best_rec']

577        3
373        4
356        3
27       3–4
459        3
        ... 
847     none
983        2
754        2
1164       5
896        7
Name: no_of_players_best_rec, Length: 1199, dtype: object

In [62]:
for i in ['Elasund', 'Elements', 'Beep!', 'Oasis','13 Days', 'Passtally', 'Portrayal', 'Trump', 'Small Start']:
    for j in df[1]:
        if i in j:
            print(i)

13 Days
